In [2]:
#Import of main libraries 
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np 
from __future__ import division, print_function
from pandas import read_excel
from pandas import DataFrame
from pandas import ExcelWriter
from pandas import ExcelFile

In [3]:
#Import of the pyomo module
from pyomo.environ import *
#Creation of a Concrete Model
model = ConcreteModel()

In [4]:
BidsDf = read_excel('Logistics Tendering Data for CaseStudy PIC2018.xlsx', sheet_name='Bids')

In [5]:
LanesDf = read_excel('Logistics Tendering Data for CaseStudy PIC2018.xlsx', sheet_name='Lots')
LanesDf.head()

,Lot,Lot#,# Shipments,AsIs Cost/Shipment,Spend AsIs,Lot Type,Site From,Site Cluster,Country To,Country Cluster,Incumbent,Lowest Bid,Avg Bid,Max Quote,# Bids
0,ISO_001,1,3,2686.8,8060.4,Long Tail,BK,Benelux,MX,Mexico,VTGT,2864.4,4313.127273,6074.4,11
1,ISO_002,2,3,2362.8,7088.4,Long Tail,BK,Benelux,MX,Mexico,INTT,2337.6,3478.690909,4942.8,11
2,ISO_003,3,2,3092.4,6184.8,Long Tail,BK,Benelux,TR,Mediterranean,MSLO,2456.4,3809.550000,5070.0,8
3,ISO_004,4,1,2755.2,2755.2,Long Tail,BK,Benelux,IL,Mediterranean,MSLO,2214.0,2856.720000,3540.0,5
4,ISO_005,5,6,2624.4,15746.4,Focus,BK,Benelux,IL,Mediterranean,BLKH,2143.2,3157.600000,4057.2,12


In [6]:
LanesDf.loc[BidsDf.loc[0,'Lot#'],'Incumbent'] #How to locate incumbent for a lane

'INTT'

In [9]:
#Create a vector indicating the bid number in which the incumbent placed a bid (Note the sum should be = the total number of lanes 234)
IncumbentVector = np.zeros(len(BidsDf.index))

for i in BidsDf.index:
    if BidsDf.loc[i,'Carrier'] == LanesDf.loc[BidsDf.loc[i,'Lot#']-1,'Incumbent']:
        IncumbentVector[i] = 1
    else:
        IncumbentVector[i] = 0


In [10]:
#Create the data frame for Incumbents
IncumbentDf = DataFrame(IncumbentVector)
IncumbentDf.columns = ['IncumbentBid']


In [15]:
gammaDf = DataFrame(np.zeros((len(LanesDf.index), len(BidsDf.index))))

for i in BidsDf.index:
    gammaDf.at[BidsDf.loc[i,'Lot#'] - 1, i] = IncumbentDf.loc[i]

In [17]:
#Number of bids in the program
model.numBids = len(BidsDf.index)
#Number of lanes in the program
model.numItems = len(LanesDf.index)

In [18]:
## Define sets
model.BIDS = Set(initialize = BidsDf.index.values)
model.LANES = Set(initialize = LanesDf.index.values)

In [19]:
# Create a dictionary of the bid values
bidValues = dict()
for bid in BidsDf.index:
    bidValues[bid] = BidsDf.loc[bid, 'Cost']
#bidValues

{0: 2864.4,
 1: 2970.0,
 2: 3225.6,
 3: 3499.2,
 4: 3915.6,
 5: 4128.0,
 6: 4166.4,
 7: 5336.4,
 8: 5528.4,
 9: 5736.0,
 10: 6074.4,
 11: 2337.6,
 12: 2515.2,
 13: 2596.7999999999997,
 14: 2733.6,
 15: 3210.0,
 16: 3270.0,
 17: 3471.6,
 18: 4138.8,
 19: 4314.0,
 20: 4735.2,
 21: 4942.8,
 22: 2456.4,
 23: 3270.0,
 24: 3462.0,
 25: 3558.0,
 26: 3960.0,
 27: 4260.0,
 28: 4440.0,
 29: 5070.0,
 30: 2214.0,
 31: 2838.0,
 32: 2842.7999999999997,
 33: 2848.7999999999997,
 34: 3540.0,
 35: 2143.2,
 36: 2378.4,
 37: 2720.4,
 38: 2842.7999999999997,
 39: 2958.0,
 40: 3007.2,
 41: 3264.0,
 42: 3445.2,
 43: 3642.0,
 44: 3714.0,
 45: 3718.7999999999997,
 46: 4057.2,
 47: 2718.0,
 48: 2720.4,
 49: 2816.4,
 50: 2958.0,
 51: 3013.2,
 52: 3445.2,
 53: 3642.0,
 54: 3714.0,
 55: 3718.7999999999997,
 56: 3723.6,
 57: 3991.2,
 58: 4713.599999999999,
 59: 2378.4,
 60: 2720.4,
 61: 2914.7999999999997,
 62: 2958.0,
 63: 3188.4,
 64: 3264.0,
 65: 3280.7999999999997,
 66: 3445.2,
 67: 3554.4,
 68: 3718.799999999

In [20]:
#Initialize bidValue parameter with the value of each bid 
model.bidValue = Param(model.BIDS, initialize = bidValues, doc='Value of each bid in the program')

In [21]:
#Create a dictionary of the total volume on each lane
demandValues = dict()
for lane in LanesDf.index:
    demandValues[lane] = 1

In [22]:
#Initialize demand parameter with the total volume on each lane (rep as demand/multiplicity)
model.demand = Param(model.LANES, initialize = demandValues, doc='Total demand on each lane')

In [23]:
#Create a dictionary of the gamma values
gamma = dict()
for lane in LanesDf.index:
    for bid in BidsDf.index:
        gamma[(lane,bid)] = gammaDf.loc[lane,bid]

In [24]:
#Initialize the gamma parameter
model.gamma = Param(model.LANES, model.BIDS, initialize=gamma, doc='gamma gives information regarding which bids are in a incumbent bids')

In [26]:
#Create a dictionary for the cardinality of each bid
cardinality = dict()
for bid in BidsDf.index:
    cardinality[bid] = gammaDf[bid].sum()
#cardinality

In [27]:
#Initialize the cardinality parameter
model.cardinality = Param(model.BIDS, initialize = cardinality, doc='Number of lanes in a bid package, |S_b|')

In [28]:
#Define the decision variable
model.x = Var(model.BIDS, domain = Binary, doc='Decision variable for each bid in the program')

In [29]:
#Objective minimizes the sum of x_b * v_b over all bids
def obj_expression(model):
    return sum(model.bidValue[i]*model.x[i] for i in model.BIDS)
model.OBJ = Objective(rule=obj_expression, sense=minimize, doc='Objective function definition')

In [30]:
#Define constraints

#def constraint_rule(model, l):
 #   return sum(model.delta[l,b]*model.x[b] for b in model.BIDS) <= 1
#model.xConstraint = Constraint(model.LANES, rule=constraint_rule)

def incumbent_constraint_rule(model, l):
    return sum(model.gamma[l,b]*model.x[b] for b in model.BIDS) <= 1
model.xConstraint = Constraint(model.LANES, rule=incumbent_constraint_rule)

def demand_constraint_rule(model):
    return sum(model.x[b]*model.cardinality[b] for b in model.BIDS) >= model.numItems
model.demandConstraint = Constraint(rule=demand_constraint_rule)

In [31]:
#Display of the output in order to retrieve and use in python
def pyomo_postprocess(options=None, instance=None, results=None):
    model.x.display()

In [32]:
#Run the model
from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("glpk")
%timeit results = opt.solve(model)

98.8 ms ± 6.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
results = opt.solve(model)
model.solutions.store_to(results)
print(results)


Problem: 
- Name: unknown
  Lower bound: 737770.8
  Upper bound: 737770.8
  Number of objectives: 1
  Number of constraints: 236
  Number of variables: 2200
  Number of nonzeros: 469
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.07663083076477051
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: optimal
  Message: None
  Objective:
    OBJ:
      Value: 737770.8000000003
  Variable:
    x[1007]:
      Value: 1
    x[1022]:
      Value: 1
    x[1032]:
      Value: 1
    x[1036]:
      Value: 1
    x[1050]:
      Value: 1
    x[1059]:
      Value: 1
    x[1072]:
      Value: 1
    x[1075]:
      Value: 1
    x[1082]:
      Value: 1
    x[108]:
      Value: 1
    x[1091]:
      Value: 1
    x[1100]:
      Value: 1
    x[1108]:
      Value: 1
    x[1122]:
      Value: 1
    x[1136]:


In [34]:
#sends results to stdout
results.write()
print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, model, results)

In [35]:
model.pprint()

In [36]:
winningBids = []
index = 0
bidNum = 0
for bids in range(2199):
    if model.x[bids].value > 0:
        winningBids.append(bidNum)
    bidNum += 1
    index += 1

In [37]:
len(winningBids)

234

In [38]:
winningBidsDf = BidsDf.iloc[winningBids]
winningBidsDf.head()

,Bid,Carrier,Lot,Lot#,Cost,Rank,Lowest Bid,Delta vs Lowest,# Shipments/Lot
2,VTGT@ISO_001,VTGT,ISO_001,1,3225.6,3,2864.4,361.2,3
11,INTT@ISO_002,INTT,ISO_002,2,2337.6,1,2337.6,0.0,3
25,MSLO@ISO_003,MSLO,ISO_003,3,3558.0,4,2456.4,1101.6,2
33,MSLO@ISO_004,MSLO,ISO_004,4,2848.8,4,2214.0,634.8,1
38,BLKH@ISO_005,BLKH,ISO_005,5,2842.8,4,2143.2,699.6,6


In [39]:
Total = winningBidsDf['Cost'].sum()
Total

737770.8

In [40]:
sum(winningBidsDf['Cost']*winningBidsDf['# Shipments/Lot'])

11430865.2

In [41]:
winningBidsDf.to_csv('IncumbentWinningBids.csv')